# Data cleaning and preprocessing
Prepare the sampled time-series dataset so that:
- types are correct (dates, categorical vs numeric),
- values are valid (no weird infinities),
- we understand missingness,
- we remove truly useless columns,
- and we don’t do aggressive imputation yet

In [1]:
import pandas as pd
import numpy as np

data_path = "amex_step4_data_sampled.parquet"
labels_path = "amex_step4_labels.parquet"

data_df = pd.read_parquet(data_path)
labels_df = pd.read_parquet(labels_path)

print(data_df.shape)
print(labels_df.shape)

data_df.head()

(361460, 190)
(30000, 2)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


Basic structural checks

In [2]:
# No missing customer IDs
print("Missing customer_ID:", data_df["customer_ID"].isna().sum())

# Label uniqueness and alignment
print("Unique customers in data:", data_df["customer_ID"].nunique())
print("Unique customers in labels:", labels_df["customer_ID"].nunique())

# Check if the customer sets match
data_customers = set(data_df["customer_ID"].unique())
label_customers = set(labels_df["customer_ID"].unique())
print("Customer sets match:", data_customers == label_customers)

Missing customer_ID: 0
Unique customers in data: 30000
Unique customers in labels: 30000
Customer sets match: True


In [3]:
# Convert date column to datetime

DATE_COL = "S_2"

data_df[DATE_COL] = pd.to_datetime(data_df[DATE_COL], errors="coerce")

# Check date parsing issues
print("Missing/invalid dates after parsing:", data_df[DATE_COL].isna().sum())

# Sort deterministically (helps debugging + later aggregation)
data_df = data_df.sort_values(["customer_ID", DATE_COL]).reset_index(drop=True)

Missing/invalid dates after parsing: 0


In [4]:
# Identifying categorical vs numeric columns

id_cols = ["customer_ID", DATE_COL]

cat_cols = [c for c in data_df.columns if data_df[c].dtype == "object" and c not in id_cols]
num_cols = [c for c in data_df.columns if c not in id_cols + cat_cols]

print("Categorical columns:", cat_cols)
print("Number of numeric columns:", len(num_cols))

Categorical columns: ['D_63', 'D_64']
Number of numeric columns: 186


In financial data, missingness can be informative — but columns that are almost always missing are often noise or unusable.

In [5]:
# Missingness report

missing_pct = (data_df.isna().mean() * 100).sort_values(ascending=False)
missing_summary = missing_pct.to_frame("missing_%")

missing_summary.head(15)

,missing_%
D_87,99.935263
D_88,99.898467
D_108,99.462458
D_111,99.458585
D_110,99.458585
B_39,99.409617
D_73,99.041664
B_42,98.694738
D_134,96.453273
D_138,96.453273


In [7]:
# Extreme missingness, over 95%

high_missing = missing_summary[missing_summary["missing_%"] > 95]
print("Columns with >95% missing:", high_missing.shape[0])
high_missing.head(20)

Columns with >95% missing: 13


,missing_%
D_87,99.935263
D_88,99.898467
D_108,99.462458
D_111,99.458585
D_110,99.458585
B_39,99.409617
D_73,99.041664
B_42,98.694738
D_134,96.453273
D_138,96.453273


We are going to drop extremely-missing columns, we'll start at 99% but we are open to modify if needed

In [8]:
MISSING_DROP_THRESHOLD = 0.99  # 99%

cols_to_drop_missing = missing_pct[missing_pct > (MISSING_DROP_THRESHOLD * 100)].index.tolist()
cols_to_drop_missing = [c for c in cols_to_drop_missing if c not in id_cols]

print("Dropping (too missing):", len(cols_to_drop_missing))

Dropping (too missing): 7


# Drop constant columns (no variance)

Constant features add no predictive power.

In [9]:
# Constant check (ignoring NaNs)
constant_cols = []
for c in num_cols:
    nun = data_df[c].nunique(dropna=True)
    if nun <= 1:
        constant_cols.append(c)

print("Constant numeric columns:", len(constant_cols))

Constant numeric columns: 1


In [10]:
# We apply drops

drop_cols = sorted(set(cols_to_drop_missing + constant_cols))
data_df = data_df.drop(columns=drop_cols)

# Recompute columns after drop
cat_cols = [c for c in data_df.columns if data_df[c].dtype == "object" and c not in id_cols]
num_cols = [c for c in data_df.columns if c not in id_cols + cat_cols]

print("New shape:", data_df.shape)

New shape: (361460, 183)


In [11]:
# We fix invalid numeric values (inf / -inf)

# Replace inf values with NaN (models and aggregation can handle NaN)
data_df[num_cols] = data_df[num_cols].replace([np.inf, -np.inf], np.nan)

# Confirm no infinities remain
print("Any infinities left:", np.isinf(data_df[num_cols].to_numpy()).any())

Any infinities left: False


Given the size of the dataset we want to make it more smooth so we can run the whole analysis without any big complications, so we will optimize memory

In [12]:
# We downcast our float64 to float32

for c in num_cols:
    if data_df[c].dtype == "float64":
        data_df[c] = data_df[c].astype(np.float32)

In [13]:
# for categorical columns we cast 'category'

for c in cat_cols:
    data_df[c] = data_df[c].astype("category")

In [15]:
# Quick check
mem_mb = data_df.memory_usage(deep=True).sum() / 1e6
print(f"Memory usage after optimization: {mem_mb:.2f} MB")


Memory usage after optimization: 304.71 MB


Now we handle categorical missing values

In [16]:
for c in cat_cols:
    data_df[c] = data_df[c].cat.add_categories(["missing"]).fillna("missing")

# We save the clean dataset

In [17]:
data_df.to_parquet("amex_step5_cleaned.parquet", index=False)
labels_df.to_parquet("amex_step5_labels.parquet", index=False)

The dataset was prepared for feature engineering by validating customer-label alignment, converting the statement date to a datetime format, quantifying missingness, removing columns with extreme missingness or no variance, and resolving invalid numeric values (±∞). Numeric dtypes were optimized for memory efficiency, and missing categorical values were explicitly labeled to preserve informative missingness patterns. The cleaned dataset was then saved to ensure reproducibility for subsequent aggregation and modeling.

# Feature engineering and temporal aggregation

Convert time-series customer histories into a single fixed-length feature vector per customer by computing aggregated statistics over time.

This preserves historical behavior while producing a tabular dataset that works with Logistic Regression, Decision Trees, Random Forest, etc.

In [18]:

data_path = "amex_step5_cleaned.parquet"
labels_path = "amex_step5_labels.parquet"

df = pd.read_parquet(data_path)
labels_df = pd.read_parquet(labels_path)

print(df.shape, labels_df.shape)
df.head()

(361460, 183) (30000, 2)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [20]:
# We define ID and date columns

ID_COL = "customer_ID"
DATE_COL = "S_2"   # statement date

# Safety: ensure date is datetime (should already be)
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

# Sort (important for "last" features)
df = df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)

In [21]:
# Identify numerical and categorical columns

id_cols = [ID_COL, DATE_COL]

cat_cols = [c for c in df.columns if str(df[c].dtype) in ["category", "object"] and c not in id_cols]
num_cols = [c for c in df.columns if c not in id_cols + cat_cols]

print("Numeric columns:", len(num_cols))
print("Categorical columns:", cat_cols)

Numeric columns: 179
Categorical columns: ['D_63', 'D_64']


Aggregate numeric features per customer

We’ll compute:
- mean, std, min, max: it shows overall behavior
- last: most recent state (often extremely predictive)
- count of non-missing: how much info exists for that feature (missingness signal)

In [22]:
num_aggs = ["mean", "std", "min", "max", "last", "count"]

num_agg_df = df.groupby(ID_COL)[num_cols].agg(num_aggs)

# Flatten MultiIndex columns: feature_stat
num_agg_df.columns = [f"{col}_{stat}" for col, stat in num_agg_df.columns]
num_agg_df = num_agg_df.reset_index()

num_agg_df.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,P_2_count,D_39_mean,D_39_std,D_39_min,...,D_144_min,D_144_max,D_144_last,D_144_count,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,D_145_count
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,13,0.010704,0.024444,0.001082,...,0.000610,0.009616,0.002970,13,0.005814,0.003294,0.000995,0.009827,0.008533,13
1,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0.794131,0.019167,0.765181,0.824061,0.824061,13,0.199325,0.238607,0.007104,...,0.001140,0.009998,0.006943,13,0.004331,0.002926,0.000672,0.009156,0.001446,13
2,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0.721835,0.045837,0.665003,0.828111,0.828111,13,0.195796,0.177785,0.000410,...,0.333697,0.508820,0.508820,13,0.095089,0.002369,0.091536,0.099441,0.093622,13
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.450927,0.081273,0.365222,0.646557,0.410590,13,0.032565,0.064191,0.000303,...,0.000296,0.007334,0.003298,13,0.004373,0.002519,0.000517,0.008526,0.008526,13
4,000d4be985f5eb4f752ed9c0e3e4e11261b86f00e28082...,0.834742,0.044746,0.790962,0.942415,0.826025,13,0.197435,0.251198,0.001556,...,0.000357,0.008306,0.005343,13,0.006329,0.003060,0.000830,0.009599,0.007470,13


# Aggregate categorical features

For categorical, two strong options are:
- last (most recent category)
- nunique (how many different categories appeared over time → stability/volatility)

Categorical like “product type” or “status code” can matter, and nunique can represent customer stability.

In [23]:
# Aggregate categorical features

cat_agg_df = None

if len(cat_cols) > 0:
    cat_aggs = ["last", "nunique"]
    cat_agg_df = df.groupby(ID_COL)[cat_cols].agg(cat_aggs)

    cat_agg_df.columns = [f"{col}_{stat}" for col, stat in cat_agg_df.columns]
    cat_agg_df = cat_agg_df.reset_index()

    cat_agg_df.head()

In [24]:
# We merge numeric and categorical aggregated features

features_df = num_agg_df

if cat_agg_df is not None:
    features_df = features_df.merge(cat_agg_df, on=ID_COL, how="left")

print(features_df.shape)
features_df.head()

(30000, 1079)


,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,P_2_count,D_39_mean,D_39_std,D_39_min,...,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,D_145_count,D_63_last,D_63_nunique,D_64_last,D_64_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,13,0.010704,0.024444,0.001082,...,0.005814,0.003294,0.000995,0.009827,0.008533,13,CR,1,O,1
1,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0.794131,0.019167,0.765181,0.824061,0.824061,13,0.199325,0.238607,0.007104,...,0.004331,0.002926,0.000672,0.009156,0.001446,13,CO,1,R,1
2,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0.721835,0.045837,0.665003,0.828111,0.828111,13,0.195796,0.177785,0.000410,...,0.095089,0.002369,0.091536,0.099441,0.093622,13,CR,1,U,2
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.450927,0.081273,0.365222,0.646557,0.410590,13,0.032565,0.064191,0.000303,...,0.004373,0.002519,0.000517,0.008526,0.008526,13,CL,1,O,1
4,000d4be985f5eb4f752ed9c0e3e4e11261b86f00e28082...,0.834742,0.044746,0.790962,0.942415,0.826025,13,0.197435,0.251198,0.001556,...,0.006329,0.003060,0.000830,0.009599,0.007470,13,CO,1,O,2


We merged the aggregated customer-level features with the training labels using customer_ID as the key. The merge is validated as one-to-one to ensure each customer appears exactly once and receives a single target label. The identifier is retained for traceability and integrity checks in later steps.

In [29]:
# Merge with labels
ID_COL = "customer_ID"
TARGET_COL = "target"

# Merge (customer_ID stays as a normal column)
model_df = features_df.merge(
    labels_df[[ID_COL, TARGET_COL]],
    on=ID_COL,
    how="inner",
    validate="one_to_one"  # ensures 1 row per customer on both sides
)

print("After merge:", model_df.shape)
print("Unique customers:", model_df[ID_COL].nunique())
model_df.head()

After merge: (30000, 1080)
Unique customers: 30000


,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,P_2_count,D_39_mean,D_39_std,D_39_min,...,D_145_std,D_145_min,D_145_max,D_145_last,D_145_count,D_63_last,D_63_nunique,D_64_last,D_64_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,13,0.010704,0.024444,0.001082,...,0.003294,0.000995,0.009827,0.008533,13,CR,1,O,1,0
1,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0.794131,0.019167,0.765181,0.824061,0.824061,13,0.199325,0.238607,0.007104,...,0.002926,0.000672,0.009156,0.001446,13,CO,1,R,1,0
2,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0.721835,0.045837,0.665003,0.828111,0.828111,13,0.195796,0.177785,0.000410,...,0.002369,0.091536,0.099441,0.093622,13,CR,1,U,2,0
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.450927,0.081273,0.365222,0.646557,0.410590,13,0.032565,0.064191,0.000303,...,0.002519,0.000517,0.008526,0.008526,13,CL,1,O,1,0
4,000d4be985f5eb4f752ed9c0e3e4e11261b86f00e28082...,0.834742,0.044746,0.790962,0.942415,0.826025,13,0.197435,0.251198,0.001556,...,0.003060,0.000830,0.009599,0.007470,13,CO,1,O,2,0


Handle missing values created by aggregation

Some features may have std = NaN if only one observation existed.

We do a simple, safe approach:
- replace infinite values (should be none)
- fill numeric NaNs with median (or 0)

Aggregation can produce missing values (e.g., standard deviation is undefined when only one observation exists). To ensure model compatibility, we replaced invalid infinities with NaN and applied median imputation to numeric aggregated features. Customer identifiers and the target were excluded from imputation.

In [30]:
# Handling missing values

# Identify numeric feature columns (exclude target and customer_ID)
feature_cols = [c for c in model_df.columns if c not in [ID_COL, TARGET_COL]]

num_feature_cols = model_df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

# Replace inf with NaN for numeric feature columns
model_df[num_feature_cols] = model_df[num_feature_cols].replace([np.inf, -np.inf], np.nan)

# Median imputation only for numeric feature columns
medians = model_df[num_feature_cols].median()
model_df[num_feature_cols] = model_df[num_feature_cols].fillna(medians)

print("Missing values remaining in numeric features:",
      model_df[num_feature_cols].isna().sum().sum())

Missing values remaining in numeric features: 0


In [31]:
# We encode categorical features

# Identify categorical columns among features (exclude ID and target)
cat_feature_cols = model_df[feature_cols].select_dtypes(include=["object", "category"]).columns.tolist()

print("Categorical feature cols to encode:", cat_feature_cols[:10], "..." if len(cat_feature_cols) > 10 else "")

# One-hot encode ONLY those columns
if len(cat_feature_cols) > 0:
    model_df = pd.get_dummies(model_df, columns=cat_feature_cols, drop_first=True)

# Confirm customer_ID survived
assert ID_COL in model_df.columns, "customer_ID was dropped during encoding!"
print("After encoding:", model_df.shape)

Categorical feature cols to encode: ['D_63_last', 'D_64_last'] 
After encoding: (30000, 1088)


In [32]:
# customer_ID checks
print("customer_ID present:", ID_COL in model_df.columns)
print("Duplicate customer_ID rows:", model_df.duplicated(ID_COL).sum())
print("Unique customers:", model_df[ID_COL].nunique())

# target checks
print("Missing targets:", model_df[TARGET_COL].isna().sum())
print("Target distribution:\n", model_df[TARGET_COL].value_counts(normalize=True))


customer_ID present: True
Duplicate customer_ID rows: 0
Unique customers: 30000
Missing targets: 0
Target distribution:
 target
0    0.737233
1    0.262767
Name: proportion, dtype: float64


In [33]:
# Save the output
model_df.to_parquet("amex_step6_customer_level.parquet", index=False)

Customer-level features were created by aggregating each customer’s time-series statement history into a fixed-length vector. Numeric variables were summarized using statistics such as mean, standard deviation, minimum, maximum, count of observed values, and the most recent (“last”) value to capture both overall behavior and recency. Categorical variables were summarized using the most recent category and the number of unique categories observed over time. The aggregated features were merged with the customer-level labels to form a single-row-per-customer modeling table suitable for classical machine-learning algorithms.

The aggregated customer-level dataset was regenerated to ensure that customer identifiers were retained for integrity checks in subsequent steps. The final dataset was saved as amex_step6_customer_level.parquet.

In [35]:
model_df.dtypes.value_counts()


float32    890
int64      185
bool        10
float64      2
object       1
Name: count, dtype: int64